In [5]:
import six.moves.cPickle as pickle
import numpy as np
import string
import re
import random
import math
import pandas as pd
from collections import Counter
from itertools import *
import tqdm

In [6]:
relation_f = ["l_embedding.txt","f_embedding.txt", "i_embedding.txt","c_embedding.txt"]

#store relational data 
for i in range(len(relation_f)):
    f_name = relation_f[i]
    max_id = 0
    with open('../data/2007_1_1/'+f_name, "r") as neigh_f:
        for line in islice(neigh_f, 1, None):
            values = line.split()
            index = int(values[0])
            if index>max_id:
                max_id = index
    print(f_name[0], max_id+1)

l 22309
f 6498
i 90024
c 45906


In [7]:
L_n, F_n, I_n, C_n = 22309, 6498, 90024, 45906

In [8]:
# 构建节点的一阶邻居
l_f_list_train = [[] for k in range(L_n)]
f_l_list_train = [[] for k in range(F_n)]
f_i_list_train = [[] for k in range(F_n)] 
i_f_list_train = [[] for k in range(I_n)] 
i_c_list_train = [[] for k in range(I_n)]
c_i_list_train = [[] for k in range(C_n)]
c_c_list_train = [[] for k in range(C_n)]

relation_f = ["l_f_list_train.txt", "f_l_list_train.txt",\
		 "f_i_list_train.txt", "i_f_list_train.txt",\
		 "i_c_list_train.txt","c_i_list_train.txt",\
		 "c_c_list_train.txt"]

#store relational data 
for i in range(len(relation_f)):
    f_name = relation_f[i]
    with open('../data/2007_1_1/'+f_name, "r") as neigh_f:
        for line in neigh_f:
            line = line.strip()
            node_id = int(re.split(':', line)[0])
            neigh_list = re.split(':', line)[1]
            # if node_id == 29956:
            #     print(neigh_list)
            neigh_list_id = re.split(',', neigh_list)
            for j in range(len(neigh_list_id)):
                rela_list_train = eval(f_name[:-4])
                # rela_list_train[node_id].append(f_name[2]+str(neigh_list_id[j]))
                list_train_set = set(rela_list_train[node_id])
                list_train_set.add(f_name[2]+str(neigh_list_id[j]))
                rela_list_train[node_id] = list(list_train_set)

In [9]:
l_neigh_list_train = [[] for k in range(L_n)]
for i in range(L_n):
    l_neigh_list_train[i] += l_f_list_train[i]
    
f_neigh_list_train = [[] for k in range(F_n)]
for i in range(F_n):
    f_neigh_list_train[i] += f_l_list_train[i]
    f_neigh_list_train[i] += f_i_list_train[i] 
    
i_neigh_list_train = [[] for k in range(I_n)]
for i in range(I_n):
    i_neigh_list_train[i] += i_f_list_train[i]
    i_neigh_list_train[i] += i_c_list_train[i] 
    
c_neigh_list_train = [[] for k in range(C_n)]
for i in range(C_n):
    c_neigh_list_train[i] += c_i_list_train[i]
    c_neigh_list_train[i] += c_c_list_train[i] 

In [10]:
node_type = ['l','f','i','c']
list_train_list = [l_f_list_train, f_neigh_list_train, i_neigh_list_train, c_neigh_list_train]
walk_n = 10
walk_L= 30
data_path = '../data/2007_1_1/'
def gen_het_rand_walk():
	het_walk_f = open(data_path + "het_random_walk.txt", "w")
	#print len(p_neigh_list_train)
	for i in range(walk_n):
		for j in range(L_n):
			if len(l_f_list_train[j]):
				curNode = "l" + str(j)
				het_walk_f.write(curNode + " ")
				for l in range(walk_L - 1):
					node_idx = node_type.index(curNode[0])
					cur_idx = int(curNode[1:])
					# print(cur_idx)
					curNode = random.choice(list_train_list[node_idx][cur_idx])
					# print(node_idx, curNode)
					het_walk_f.write(curNode + " ")
				het_walk_f.write("\n")
	het_walk_f.close()

In [11]:
gen_het_rand_walk()

In [ ]:
# import string
# import re
# import random
# import math
# import numpy as np
from gensim.models import Word2Vec
from itertools import *
dimen = 384
window = 5


def read_random_walk_corpus():
	walks=[]
	inputfile = open("../data/2007_1_1/het_random_walk.txt", "r")
	for line in inputfile:
		path = []
		node_list=re.split(' ',line)
		for i in range(len(node_list)):
			path.append(node_list[i])
		walks.append(path)
	inputfile.close()
	return walks


walk_corpus = read_random_walk_corpus()
model = Word2Vec(walk_corpus, vector_size = dimen, window = window, min_count = 0, workers = 2, sg = 1, hs = 0, negative = 5)


print("Output...")
#model.wv.save_word2vec_format("../data/node_embedding.txt")
model.wv.save_word2vec_format("../data/2007_1_1/node_net_embedding.txt")

In [ ]:
def het_walk_restart():
    l_neigh_list_train_ = [[] for k in range(L_n)]
    f_neigh_list_train_ = [[] for k in range(F_n)]
    i_neigh_list_train_ = [[] for k in range(I_n)]
    c_neigh_list_train_ = [[] for k in range(C_n)]
    #generate neighbor set via random walk with restart
    node_n = [L_n, F_n, I_n, C_n]
    node_type = ['l','f','i','c']
    # 所有的邻居节点
    self_neigh_train_list = [l_neigh_list_train, f_neigh_list_train, i_neigh_list_train, c_neigh_list_train]
    # random walk邻居节点
    neigh_train_list = [l_neigh_list_train_, f_neigh_list_train_, i_neigh_list_train_, c_neigh_list_train_]
    list_train_list = [l_f_list_train, f_i_list_train, i_c_list_train, c_c_list_train]
    neighbor_size = 100
    all_nodes = L_n + F_n + I_n + C_n
    l_constaint = neighbor_size*1.1*(L_n/all_nodes)
    f_constaint = neighbor_size*1.1*(F_n*2/all_nodes)
    i_constaint = neighbor_size*1.1*(I_n/all_nodes)
    c_constaint = neighbor_size*1.1*(C_n/all_nodes)
    constaint_list = [l_constaint, f_constaint, i_constaint, c_constaint]
    print(constaint_list)
    for i in range(0, len(node_n)):
        for j in tqdm.tqdm(range(0,node_n[i])):
            neigh_temp = list_train_list[i][j]
            neigh_train = neigh_train_list[i][j]
            curNode = node_type[i] + str(j)
            if len(neigh_temp):
                neigh_L = 0
                l_L = 0
                f_L = 0
                i_L = 0
                c_L = 0
                node_L_list = [l_L,f_L,i_L,c_L]
                count = 0
                while (neigh_L < neighbor_size) & (count<=300): #maximum neighbor size = 100
                    rand_p = random.random() #return p
                    if rand_p > 0.5:
                        curNode_idx = node_type.index(curNode[0])
                        curNode = random.choice(self_neigh_train_list[curNode_idx][int(curNode[1:])])
                        if curNode in neigh_train:
                            count += 1
                        curNode_idx = node_type.index(curNode[0])
                        if node_L_list[curNode_idx] < constaint_list[curNode_idx]: #size constraint (make sure each type of neighobr is sampled)
                            neigh_train.append(curNode)
                            neigh_L += 1
                            node_L_list[curNode_idx] += 1
                    else:
                        curNode = node_type[i] + str(j)
    for i in range(4):
        for j in range(node_n[i]):
            neigh_train_list[i][j] = list(neigh_train_list[i][j])

    neigh_f = open(data_path + "het_neigh_train.txt", "w")
    for i in range(4):
        for j in range(node_n[i]):
            neigh_train = neigh_train_list[i][j]
            curNode = node_type[i] + str(j)
            if len(neigh_train):
                neigh_f.write(curNode + ":")
                for k in range(len(neigh_train) - 1):
                    neigh_f.write(neigh_train[k] + ",")
                neigh_f.write(neigh_train[-1] + "\n")
    neigh_f.close()
